In [2]:
# importing OpenCV library 
import cv2
from gtts import gTTS
from playsound import playsound
import os
import speech_recognition as sr


def newperson(frame):
    
    def ask_name():
        # Create a text-to-speech object
        tts = gTTS(text=f"तपाईको नाम के हो ?", lang='ne',tld='co.in',slow=False)
        # Save the audio file
        audio_file = "output.mp3"
        tts.save(audio_file)
        # Play the audio file
        playsound(audio_file)
        # Remove the audio file after playing
        os.remove(audio_file)

    ask_name()


    def name_input(max_retries=2, timeout=30):
        # Initialize recognizer
        recognizer = sr.Recognizer()

        for attempt in range(max_retries + 1):
            # Capture audio from the microphone
            with sr.Microphone() as source:
                print(f"Attempt {attempt + 1}: Listening in Nepali...")

                try:
                    # Adjust for ambient noise and listen with a timeout
                    recognizer.adjust_for_ambient_noise(source)
                    audio = recognizer.listen(source, timeout=timeout)

                    # Recognize the speech in Nepali using Google Web Speech API
                    name = recognizer.recognize_google(audio, language="ne-NP")
                    print(f"Recognized Name: {name}")
                    return name  # If successful, return the recognized name

                except sr.UnknownValueError:
                    print("Could not understand the audio, please say it again.")

                except sr.WaitTimeoutError:
                    print("Listening timed out, please try saying your name again.")

                except sr.RequestError:
                    print("Request failed; check your internet connection.")
                    break  # If there's a connection issue, break out of the loop

        # If no valid name is recognized after all attempts
        print("Failed to recognize after several attempts.")
        return name
        





    # initialize the camera 
    # If you have multiple camera connected with 
    # current device, assign a value in cam_port 
    # variable according to that 
    # cam_port = 0
    # cam = cv2.VideoCapture(cam_port)

    # while True:
    #     success, img = cam.read()
    #     #img = captureScreen()
    #     # saving image in local storage 
    
    cv2.imwrite(f"testimage/{name_input()}.png", frame)
        # # break
        # #cv2.imshow('Webcam',img)
        # #cv2.waitKey(1)
        # cv2.imshow('my video', img)
        # if cv2.waitKey(13) & 0xFF == ord('q'):
        #     cv2.destroyAllWindows()
        #     break

In [3]:
import face_recognition
import os
import csv

def append_face_encodings_to_csv(image_folder, csv_filename):
    # List to store encodings and names for new images
    face_data = []

    # Get the names of already encoded people from the CSV
    existing_names = set()
    if os.path.exists(csv_filename):
        with open(csv_filename, mode="r") as file:
            reader = csv.reader(file)
            try:
                # Try to skip the header row
                next(reader)
            except StopIteration:
                # If the file is empty, do nothing
                print(f"{csv_filename} is empty, starting fresh.")
            else:
                # Read all existing names
                for row in reader:
                    existing_names.add(row[0])  # The name is the first column

    # Loop through each image in the folder
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
            # Get the person's name from the file name (without extension)
            person_name = os.path.splitext(filename)[0]

            # Skip if the name already exists in the CSV
            if person_name in existing_names:
                print(f"Skipping {person_name}, already encoded.")
                continue

            # Load the image file
            image_path = os.path.join(image_folder, filename)
            image = face_recognition.load_image_file(image_path)

            # Get face encodings
            face_encodings = face_recognition.face_encodings(image)

            # Check if a face was found
            if len(face_encodings) > 0:
                # Use the first face encoding
                face_encoding = face_encodings[0]

                # Append the name and encoding to the list
                face_data.append([person_name] + face_encoding.tolist())
                print(f"Encoded and added {person_name}")

    # Append the new face data to the existing CSV file
    with open(csv_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        # Write the header row if the file is empty
        if os.stat(csv_filename).st_size == 0:
            writer.writerow(["Name"] + [f"Encoding_{i+1}" for i in range(128)])  # Header
        # Write each new person's name and encodings
        writer.writerows(face_data)

    if face_data:
        print(f"New face encodings appended to {csv_filename}")
    else:
        print("No new face encodings to add.")

# Example usage:
image_folder = "testimage"
csv_filename = "facedetails.csv"
append_face_encodings_to_csv(image_folder, csv_filename)


Skipping राम, already encoded.
Skipping शुभस, already encoded.
Skipping sir, already encoded.
No new face encodings to add.


In [4]:
import csv
import numpy as np

def load_face_encodings_from_csv(csv_filename):
    known_face_encodings = []
    known_face_names = []

    if os.path.exists(csv_filename):
        with open(csv_filename, mode="r") as file:
            reader = csv.reader(file)
            header = next(reader)  # Skip the header row

            for row in reader:
                name = row[0]
                encoding = list(map(float, row[1:]))  # Convert all encoding values to float

                known_face_names.append(name)
                known_face_encodings.append(np.array(encoding))

    return known_face_encodings, known_face_names

# Example usage
csv_filename = "facedetails.csv"
known_face_encodings, known_face_names = load_face_encodings_from_csv(csv_filename)

print("Known face names:", known_face_names)
print("Number of known face encodings:", known_face_encodings)


Known face names: ['sir', 'शुभस', 'राम']
Number of known face encodings: [array([-1.04551524e-01,  8.17660615e-02,  6.38923347e-02,  1.58169027e-02,
       -6.94673732e-02, -1.07918844e-01, -2.08491236e-02, -2.27949291e-01,
        7.83200413e-02, -5.57524785e-02,  2.25337625e-01, -2.37203259e-02,
       -2.60618776e-01, -1.16440207e-01,  9.29652154e-03,  1.93225726e-01,
       -2.00881451e-01, -1.81661054e-01, -1.00697257e-01, -6.61355704e-02,
       -3.66029069e-02, -5.21890493e-03,  1.24986209e-01,  4.65265587e-02,
       -7.31325746e-02, -3.08529794e-01, -6.68665767e-02, -9.03701037e-02,
        1.71684884e-02, -1.23930285e-02, -5.04123494e-02,  4.28354740e-02,
       -2.76541919e-01, -1.07292563e-01,  6.30083531e-02,  1.36473238e-01,
       -2.45850850e-02, -3.82841490e-02,  2.19216079e-01,  3.94019485e-03,
       -1.67061761e-01, -1.07953837e-02,  8.66704285e-02,  2.07236692e-01,
        2.50734478e-01,  1.20223407e-02,  4.25493009e-02, -1.26341000e-01,
        8.73768479e-02, -1

In [6]:
import face_recognition
import cv2
import numpy as np
from gtts import gTTS
from playsound import playsound
import os

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

def greet_person(name):
    # Create a text-to-speech object
    tts = gTTS(text=f"{name} नमस्ते!", lang='ne', tld='co.in', slow=False)
    # Save the audio file
    audio_file = "output.mp3"
    tts.save(audio_file)
    # Play the audio file
    playsound(audio_file)
    # Remove the audio file after playing
    os.remove(audio_file)

# Initialize variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
greeted_names = set()  # Set to keep track of greeted names

# Static bounding box for face alignment (in the middle of the screen)
static_box_start = (150, 100)  # top-left corner of the static bounding box
static_box_end = (450, 400)    # bottom-right corner of the static bounding box

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Draw the static bounding box on the frame
    cv2.rectangle(frame, static_box_start, static_box_end, (0, 255, 0), 2)

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding, face_location in zip(face_encodings, face_locations):
            # Scale up the face location
            top, right, bottom, left = [v * 4 for v in face_location]

            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

            # Check if the face is inside the static bounding box
            if (left > static_box_start[0] and right < static_box_end[0] and 
                top > static_box_start[1] and bottom < static_box_end[1]):
                
                if name == "Unknown":
                    # Capture the image and save it for a new person
                    newperson(frame)  # Custom function to handle new person
                    append_face_encodings_to_csv(image_folder='testimage', csv_filename='facedetails.csv')
                    known_face_encodings, known_face_names = load_face_encodings_from_csv('facedetails.csv')

                # Greet the detected person if not already greeted
                if name != "Unknown" and name not in greeted_names:
                    greet_person(name)
                    greeted_names.add(name)  # Add name to the set of greeted names

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Optionally, draw the person's name on the screen
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()


Attempt 1: Listening in Nepali...
Recognized Name: भेदभारी पनि हो।
Skipping राम, already encoded.
Skipping शुभस, already encoded.
Encoded and added भेदभारी पनि हो।
Skipping sir, already encoded.
New face encodings appended to facedetails.csv
